In [1]:
from numpy import __version__ as numpy_version
from pandas import __version__ as pandas_version

print(f"Numpy - {numpy_version}")
print(f"Pandas - {pandas_version}")


Numpy - 1.18.1
Pandas - 1.0.0


In [2]:
import numpy as np
import pandas as pd
#import altair as alt
import datetime as dt
from pathlib import Path


In [3]:
# local connection information
import local_db

connection = local_db.connection()


In [4]:
print(f"Last Run: {dt.datetime.now()}")

today = dt.datetime.now().strftime("%Y%m%d")
year = 2020

fall_term = f"{year}.Fall"
print(f"{fall_term}")

Last Run: 2020-07-09 07:26:53.041475
2020.Fall


In [5]:
sql_str = (
    "SELECT DISTINCT "
    + "PEOPLE_CODE_ID, "
    + "ACADEMIC_YEAR, "
    + "ACADEMIC_TERM, "
    + "FIELD_ID, "
    + "FIELD_DATE "
    + "FROM [Campus6].[dbo].[STAGEHISTORY] "
    + f"WHERE ACADEMIC_YEAR='{year}' "
    + "and ACADEMIC_TERM='FALL' "
    + "and FIELD_ID in (10,49,50,39,13) "
    + "ORDER BY PEOPLE_CODE_ID, FIELD_DATE, FIELD_ID "
)

df_stghist = pd.read_sql_query(sql_str, connection)

print(df_stghist.shape)


(1116, 5)


In [6]:
df_stghist.head()

,PEOPLE_CODE_ID,ACADEMIC_YEAR,ACADEMIC_TERM,FIELD_ID,FIELD_DATE
0,P000030533,2020,FALL,39,2020-05-06
1,P000032016,2020,FALL,13,2020-03-06
2,P000032016,2020,FALL,13,2020-05-18
3,P000048221,2020,FALL,39,2020-06-03
4,P000051896,2020,FALL,39,2020-04-24


In [7]:
df_stghist.loc[((df_stghist['FIELD_ID']==10)|(df_stghist['FIELD_ID']==50))]

,PEOPLE_CODE_ID,ACADEMIC_YEAR,ACADEMIC_TERM,FIELD_ID,FIELD_DATE
6,P000053294,2020,FALL,10,2020-06-17
7,P000053294,2020,FALL,50,2020-06-17
9,P000055045,2020,FALL,10,2020-04-09
10,P000055045,2020,FALL,50,2020-04-09
11,P000055385,2020,FALL,10,2020-05-05
...,...,...,...,...,...
1103,P000061304,2020,FALL,10,2020-06-08
1111,P000061463,2020,FALL,10,2020-06-20
1112,P000061463,2020,FALL,50,2020-06-20
1114,P000061493,2020,FALL,10,2020-07-07


### Use admissions.py code

In [8]:
# read VWSSTAGERANKING data
sql_str = (
    "SELECT STAGERANKING_ID, field_name, field_value "
    + "FROM VWSSTAGERANKING WHERE "
    + "status = 'A' "
)
stgrnk = pd.read_sql_query(sql_str, connection)

# read STAGEHISTORY data
sql_str = (
    "SELECT PEOPLE_CODE_ID, ACADEMIC_YEAR, ACADEMIC_TERM, "
    + "ACADEMIC_SESSION, FIELD_ID, FIELD_DATE, HIDDEN "
    + "FROM STAGEHISTORY WHERE "
    + "HIDDEN = 'N' "
    + f"AND ACADEMIC_YEAR = '{year}' "
    + "and ACADEMIC_TERM='FALL' "
)
stg_hist = pd.read_sql_query(sql_str, connection)

stg_hist = stg_hist.rename(columns={"FIELD_DATE": "create_date"})
stage_data = pd.merge(
    stg_hist, stgrnk, left_on=["FIELD_ID"], right_on=["STAGERANKING_ID"], how="left"
)
keep_fields = [
    "PEOPLE_CODE_ID",
    "ACADEMIC_YEAR",
    "ACADEMIC_TERM",
    "ACADEMIC_SESSION",
    "field_name",
    "field_value",
    "create_date",
]
stage_data = stage_data.loc[~stage_data["create_date"].isnull(), keep_fields]

print(stage_data.shape)

(7290, 7)


In [9]:
# read ACADEMIC data
sql_str = (
    "SELECT PEOPLE_CODE_ID, ACADEMIC_YEAR, ACADEMIC_TERM, "
    + "ACADEMIC_SESSION, POPULATION, INQUIRY_FLAG, "
    + "APPLICATION_FLAG, APPLICATION_DATE, "
    + "APP_STATUS, APP_STATUS_DATE, "
    + "APP_DECISION, APP_DECISION_DATE "
    + "FROM ACADEMIC WHERE "
    + f"ACADEMIC_YEAR >= '{year}' "
)
academic = pd.read_sql_query(sql_str, connection)

app_data = academic.loc[
    ~(academic["POPULATION"].isin(["ADVSTU", "NOND"]))
    & ((academic["INQUIRY_FLAG"] == "Y") | (academic["APPLICATION_FLAG"] == "Y"))
]

applied = app_data[app_data["APP_STATUS"].notnull()].rename(
    columns={"APP_STATUS": "field_value", "APP_STATUS_DATE": "create_date"}
)
applied.loc[:, "field_name"] = "Application Status"
applied = applied.loc[~applied["create_date"].isnull(), keep_fields]

accepted = app_data[app_data["APP_DECISION"].notnull()].rename(
    columns={"APP_DECISION": "field_value", "APP_DECISION_DATE": "create_date"}
)
accepted.loc[:, "field_name"] = "Application Decision"
accepted = accepted.loc[~accepted["create_date"].isnull(), keep_fields]


In [10]:
# stack Stage History, Academic Applied and Academic Accepted
adm_df = stage_data.append(applied).append(accepted)

adm_df = adm_df.loc[
    (
        (adm_df["ACADEMIC_TERM"].isin(["FALL", "SPRING"]))
        & (adm_df["ACADEMIC_SESSION"] == "MAIN")
        & (adm_df["ACADEMIC_YEAR"] == f"{year}")
    )
]


In [11]:
print(adm_df.shape)

(8353, 7)


In [12]:
print(adm_df.columns)

Index(['PEOPLE_CODE_ID', 'ACADEMIC_YEAR', 'ACADEMIC_TERM', 'ACADEMIC_SESSION',
       'field_name', 'field_value', 'create_date'],
      dtype='object')


In [13]:
# create new fields
adm_df["year_term"] = (
    adm_df["ACADEMIC_YEAR"] + "." + adm_df["ACADEMIC_TERM"].str.title()
)


In [14]:
adm_df = adm_df.loc[(adm_df["year_term"]==fall_term)]
print(adm_df.shape)

(8191, 8)


In [15]:
adm_keep_values = [
    "300",
    "ACC",
    "ACXL",
    "CANC",
    "DEF",
    "DEFR",
    "DENY",
    "DPAC",
    "TRDP",
    "TRPD",
    "TRNS",
    "WAIT",
    "500",
    "PEND",
    "COMP",
]
adm_keep_cols = ["PEOPLE_CODE_ID", "year_term", "create_date", "field_value"]
adm_df = adm_df.loc[(adm_df["field_value"].isin(adm_keep_values)), adm_keep_cols]


In [16]:
print(adm_df.shape)

(5778, 4)


In [17]:
adm_df.head()

,PEOPLE_CODE_ID,year_term,create_date,field_value
2,P000057181,2020.Fall,2019-12-11,300
3,P000057181,2020.Fall,2019-12-11,PEND
7,P000057181,2020.Fall,2019-11-17,500
8,P000057181,2020.Fall,2019-11-17,DPAC
10,P000057454,2020.Fall,2019-12-12,300


In [18]:
# admissions status table
admission_status = {
    "300": "1_Applied",
    "ACC": "2_Accepted",
    "ACXL": "4_Canceled",
    "CANC": "4_Canceled",
    "DEF": "4_Canceled",
    "DEFR": "4_Canceled",
    "DENY": "4_Canceled",
    "DPAC": "3_Deposited",
    "TRDP": "3_Deposited",
    "TRPD": "3_Deposited",
    "TRNS": "2_Accepted",
    "WAIT": "2_Accepted",
    "500": "3_Deposited",
    "PEND": "1_Applied",
    "COMP": "1_Applied",
}
adm_stat = pd.DataFrame(
    list(admission_status.items()), columns=["field_value", "admission_status"]
)

adm_stat

,field_value,admission_status
0,300,1_Applied
1,ACC,2_Accepted
2,ACXL,4_Canceled
3,CANC,4_Canceled
4,DEF,4_Canceled
5,DEFR,4_Canceled
6,DENY,4_Canceled
7,DPAC,3_Deposited
8,TRDP,3_Deposited
9,TRPD,3_Deposited


In [19]:
adm_df1 = (
    pd.merge(adm_df, adm_stat, on=["field_value"], how="left")
    .drop(["field_value"], axis=1)
    .drop_duplicates(
        ["PEOPLE_CODE_ID", "year_term", "create_date", "admission_status"]
    )
)

adm_df1.head()

,PEOPLE_CODE_ID,year_term,create_date,admission_status
0,P000057181,2020.Fall,2019-12-11,1_Applied
2,P000057181,2020.Fall,2019-11-17,3_Deposited
4,P000057454,2020.Fall,2019-12-12,1_Applied
6,P000057454,2020.Fall,2019-11-20,3_Deposited
8,P000057222,2020.Fall,2019-12-12,1_Applied


In [20]:
adm_df1 = adm_df1.sort_values(
    ["year_term", "PEOPLE_CODE_ID", "admission_status", "create_date"]
).drop_duplicates(["year_term", "PEOPLE_CODE_ID", "admission_status"], keep="first")


In [21]:
adm_df1 = adm_df1[["PEOPLE_CODE_ID", "admission_status", "create_date"]]

adm_df1["status"] = 1

In [22]:
print(adm_df1.shape)

(2947, 4)


In [23]:
print(adm_df1.columns)

Index(['PEOPLE_CODE_ID', 'admission_status', 'create_date', 'status'], dtype='object')


In [24]:
adm_df1.head()

,PEOPLE_CODE_ID,admission_status,create_date,status
3332,P000024911,1_Applied,2020-05-21,1
3333,P000024911,2_Accepted,2020-05-12,1
1465,P000029026,1_Applied,2020-06-15,1
2581,P000029026,2_Accepted,2020-07-01,1
1911,P000030533,1_Applied,2020-01-09,1


In [25]:
e = adm_df1.pivot_table(
    index=["PEOPLE_CODE_ID"],
    columns=["admission_status"],
    values=["status"],
)
e = e.fillna(0)

e.columns = e.columns.droplevel()


In [26]:
e

admission_status,1_Applied,2_Accepted,3_Deposited,4_Canceled
PEOPLE_CODE_ID,,,,
P000024911,1.0,1.0,0.0,0.0
P000029026,1.0,1.0,0.0,0.0
P000030533,1.0,1.0,0.0,1.0
P000032016,1.0,1.0,0.0,1.0
P000042541,1.0,0.0,0.0,0.0
...,...,...,...,...
P000061551,1.0,1.0,0.0,0.0
P000061566,1.0,0.0,0.0,0.0
P000061570,1.0,0.0,0.0,0.0


In [27]:
dep = e.loc[(e["3_Deposited"]==1)]

In [28]:
active_dep = e.loc[(e["3_Deposited"]==1)&(~(e["4_Canceled"]==1))]
print(active_dep.shape)
active_dep.head()

(222, 4)


admission_status,1_Applied,2_Accepted,3_Deposited,4_Canceled
PEOPLE_CODE_ID,,,,
P000053294,1.0,1.0,1.0,0.0
P000055045,0.0,0.0,1.0,0.0
P000055385,1.0,1.0,1.0,0.0
P000055962,0.0,0.0,1.0,0.0
P000056127,1.0,0.0,1.0,0.0


In [29]:
sql_str = (
    f"""
SELECT [PEOPLE].[PEOPLE_CODE_ID],
	[PEOPLE].[LAST_NAME],
	[PEOPLE].[FIRST_NAME],
	[PEOPLE].[MIDDLE_NAME],
	[PEOPLE].[BIRTH_DATE],
	[PEOPLE].[GOVERNMENT_ID],
	[ADDRESS].[EMAIL_ADDRESS],
	[ADDRESS].[ADDRESS_TYPE],
	[ADDRESS].[ADDRESS_LINE_1],
	[ADDRESS].[CITY],
	[ADDRESS].[STATE],
    [ADDRESS].[COUNTRY], 
	[ADDRESS].[ZIP_CODE],
	[ACADEMIC].[CREDITS],
	[ACADEMIC].[DEGREE],
	[ACADEMIC].[CURRICULUM],
	[ACADEMIC].[ACADEMIC_YEAR],
	[ACADEMIC].[ACADEMIC_TERM],
	[ACADEMIC].[ADMIT_YEAR],
	[ACADEMIC].[ADMIT_TERM],
	[ACADEMIC].[APP_STATUS_DATE],
	[ACADEMIC].[APP_STATUS],
	[ACADEMIC].[COLLEGE_ATTEND],
	[ACADEMIC].[REVISION_DATE],
	[ACADEMIC].[REVISION_TIME],
	[ACADEMIC].[REVISION_OPID]
FROM [Campus6].[dbo].[ACADEMIC], [Campus6].[dbo].PEOPLE, [Campus6].[dbo].ADDRESS
WHERE ( 
	[ACADEMIC].[PEOPLE_CODE_ID] = [ADDRESS].[PEOPLE_ORG_CODE_ID] AND
	[ACADEMIC].[PEOPLE_ID] = [PEOPLE].[PEOPLE_ID] 
	  ) AND
	  (  
	 [ACADEMIC].[ADMIT_YEAR] = N'{year}'  and
	 [ACADEMIC].[ADMIT_TERM] = N'fall'  And
	 [ACADEMIC].[ACADEMIC_YEAR] = N'{year}'  And
	 [ACADEMIC].[ACADEMIC_TERM] = N'fall'  And
	 [ADDRESS].[ADDRESS_TYPE] = N'HOME'  and
	 [ACADEMIC].[APP_STATUS] = N'500'  and
	 [ACADEMIC].[ACADEMIC_SESSION] = N''   and
     (ACADEMIC.APP_DECISION = 'DPAC' or
	  ACADEMIC.APP_DECISION = 'TRDP')
      )
--	  and (GOVERNMENT_ID is null or GOVERNMENT_ID='' or GOVERNMENT_ID=' ')
GROUP BY [PEOPLE].[PEOPLE_CODE_ID], 
         [PEOPLE].[LAST_NAME], 
		 [PEOPLE].[FIRST_NAME], 
		 [PEOPLE].[MIDDLE_NAME], 
		 [PEOPLE].[BIRTH_DATE], 
		 [PEOPLE].[GOVERNMENT_ID],
		 [ADDRESS].[EMAIL_ADDRESS], 
		 [ADDRESS].[ADDRESS_TYPE], 
		 [ADDRESS].[ADDRESS_LINE_1], 
		 [ADDRESS].[CITY], 
		 [ADDRESS].[STATE], 
		 [ADDRESS].[COUNTRY], 
		 [ADDRESS].[ZIP_CODE], 
		 [ACADEMIC].[CREDITS], 
		 [ACADEMIC].[DEGREE], 
		 [ACADEMIC].[CURRICULUM], 
		 [ACADEMIC].[ACADEMIC_YEAR], 
		 [ACADEMIC].[ACADEMIC_TERM], 
		 [ACADEMIC].[ADMIT_YEAR],
		 [ACADEMIC].[ADMIT_TERM], 
		 [ACADEMIC].[APP_STATUS_DATE], 
		 [ACADEMIC].[APP_STATUS],
		 [ACADEMIC].[COLLEGE_ATTEND],
		 [ACADEMIC].[REVISION_DATE],
		 [ACADEMIC].[REVISION_TIME],
		 [ACADEMIC].[REVISION_OPID]


ORDER BY 
--		[ACADEMIC].[APP_STATUS_DATE],
		[PEOPLE].[LAST_NAME]
--		[ACADEMIC].[REVISION_DATE],
--	    [ACADEMIC].[REVISION_TIME]
    """
)

df_deposits = pd.read_sql_query(sql_str, connection)

print(df_deposits.shape)
df_deposits.head()

(222, 26)


,PEOPLE_CODE_ID,LAST_NAME,FIRST_NAME,MIDDLE_NAME,BIRTH_DATE,GOVERNMENT_ID,EMAIL_ADDRESS,ADDRESS_TYPE,ADDRESS_LINE_1,CITY,...,ACADEMIC_YEAR,ACADEMIC_TERM,ADMIT_YEAR,ADMIT_TERM,APP_STATUS_DATE,APP_STATUS,COLLEGE_ATTEND,REVISION_DATE,REVISION_TIME,REVISION_OPID
0,P000057892,Ackerman,Annaliese,Rose,2001-09-27,180802279,aackerman@paulsmiths.edu,HOME,63 Sage Dr,Pottstown,...,2020,FALL,2020,FALL,2019-12-10,500,FTF,2020-07-08,1900-01-01 10:48:02.077,CARLINJ
1,P000059248,Adams,Gabrielle,M,2002-03-28,058929892,gadams@paulsmiths.edu,HOME,4134 Arquint Rd,Vernon Center,...,2020,FALL,2020,FALL,2020-02-27,500,FTF,2020-07-08,1900-01-01 10:48:23.667,CARLINJ
2,P000060871,Ahlers,Saphire,,1998-09-09,533416747,sahlers@paulsmiths.edu,HOME,1102 Glenwood Boulevard,Schenectady,...,2020,FALL,2020,FALL,2020-04-29,500,TRAN,2020-06-29,1900-01-01 09:20:20.107,CARLINJ
3,P000056509,Akabati,Azanui,Stanley,1999-03-04,999956509,aakabati@paulsmiths.edu,HOME,Below Ccast Bambili,"Bamenda, Cameroon",...,2020,FALL,2020,FALL,2020-01-06,500,FTF,2020-06-23,1900-01-01 13:41:22.827,CARLINJ
4,P000059379,Akanji,Habeeb Ademola,,1997-05-31,,hakanji@paulsmiths.edu,HOME,N0 25,Ibadan,...,2020,FALL,2020,FALL,2020-02-19,500,FTF,2020-07-06,1900-01-01 14:44:18.397,CARLINJ


In [30]:
cmp = (
    pd.merge(active_dep, df_deposits, on=["PEOPLE_CODE_ID"], how="outer", indicator=True)
)

print(cmp.shape)
cmp

(222, 31)


,PEOPLE_CODE_ID,1_Applied,2_Accepted,3_Deposited,4_Canceled,LAST_NAME,FIRST_NAME,MIDDLE_NAME,BIRTH_DATE,GOVERNMENT_ID,...,ACADEMIC_TERM,ADMIT_YEAR,ADMIT_TERM,APP_STATUS_DATE,APP_STATUS,COLLEGE_ATTEND,REVISION_DATE,REVISION_TIME,REVISION_OPID,_merge
0,P000053294,1.0,1.0,1.0,0.0,Kwasniak,Paulina,Aldona,1999-10-27,120888355,...,FALL,2020,FALL,2020-06-17,500,TRAN,2020-07-06,1900-01-01 15:37:06.133,CARLINJ,both
1,P000055045,0.0,0.0,1.0,0.0,Compo,Michael,Thomas,2000-08-03,068903799,...,FALL,2020,FALL,2020-04-09,500,TRAN,2020-06-23,1900-01-01 13:47:57.593,CARLINJ,both
2,P000055385,1.0,1.0,1.0,0.0,Poirier,Sophia,Rose,1999-12-28,651129737,...,FALL,2020,FALL,2020-05-05,500,TRAN,2020-07-06,1900-01-01 14:46:02.833,CARLINJ,both
3,P000055962,0.0,0.0,1.0,0.0,Farra,Lina,,2001-06-28,119900456,...,FALL,2020,FALL,2020-04-29,500,TRAN,2020-06-23,1900-01-01 13:12:46.370,CARLINJ,both
4,P000056127,1.0,0.0,1.0,0.0,Bashaw,Brittany,,2002-01-31,052926132,...,FALL,2020,FALL,2019-12-15,500,FTF,2020-06-29,1900-01-01 08:57:50.250,CARLINJ,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,P000061257,1.0,1.0,1.0,0.0,Bennett,Hannah,Marie,1997-08-16,111867487,...,FALL,2020,FALL,2020-06-01,500,TRAN,2020-06-23,1900-01-01 13:16:41.610,CARLINJ,both
218,P000061287,1.0,1.0,1.0,0.0,Russell,Vincent,Mark,2002-03-08,012865655,...,FALL,2020,FALL,2020-06-09,500,FTF,2020-06-10,1900-01-01 16:07:00.373,DKOWAL,both
219,P000061304,1.0,1.0,1.0,0.0,Avant,Kalle,Alicia,2001-02-08,,...,FALL,2020,FALL,2020-06-08,500,FTF,2020-06-29,1900-01-01 09:00:52.140,CARLINJ,both
220,P000061463,1.0,1.0,1.0,0.0,Fletcher,Avrielle,,2001-03-17,095900217,...,FALL,2020,FALL,2020-06-20,500,TRAN,2020-06-29,1900-01-01 09:01:36.250,CARLINJ,both


In [31]:
cmp.loc[(cmp["_merge"]=="left_only")]

,PEOPLE_CODE_ID,1_Applied,2_Accepted,3_Deposited,4_Canceled,LAST_NAME,FIRST_NAME,MIDDLE_NAME,BIRTH_DATE,GOVERNMENT_ID,...,ACADEMIC_TERM,ADMIT_YEAR,ADMIT_TERM,APP_STATUS_DATE,APP_STATUS,COLLEGE_ATTEND,REVISION_DATE,REVISION_TIME,REVISION_OPID,_merge


In [32]:
cmp.loc[(cmp["_merge"]=="right_only")]

,PEOPLE_CODE_ID,1_Applied,2_Accepted,3_Deposited,4_Canceled,LAST_NAME,FIRST_NAME,MIDDLE_NAME,BIRTH_DATE,GOVERNMENT_ID,...,ACADEMIC_TERM,ADMIT_YEAR,ADMIT_TERM,APP_STATUS_DATE,APP_STATUS,COLLEGE_ATTEND,REVISION_DATE,REVISION_TIME,REVISION_OPID,_merge


In [33]:
adm_df1.loc[(adm_df1["PEOPLE_CODE_ID"]=="P000057537")]

,PEOPLE_CODE_ID,admission_status,create_date,status
177,P000057537,1_Applied,2019-12-23,1
179,P000057537,3_Deposited,2019-12-19,1
1598,P000057537,4_Canceled,2020-06-26,1


In [34]:
df_deposits.loc[(df_deposits["PEOPLE_CODE_ID"]=="P000057537")]

,PEOPLE_CODE_ID,LAST_NAME,FIRST_NAME,MIDDLE_NAME,BIRTH_DATE,GOVERNMENT_ID,EMAIL_ADDRESS,ADDRESS_TYPE,ADDRESS_LINE_1,CITY,...,ACADEMIC_YEAR,ACADEMIC_TERM,ADMIT_YEAR,ADMIT_TERM,APP_STATUS_DATE,APP_STATUS,COLLEGE_ATTEND,REVISION_DATE,REVISION_TIME,REVISION_OPID
100,P000057537,Hollenbeck,Madison,Rose,2002-09-18,084923796,mhollenbeck@paulsmiths.edu,HOME,94 Mclaughlin Ave,Tupper Lake,...,2020,FALL,2020,FALL,2020-06-29,500,FTF,2020-06-29,1900-01-01 07:58:48.960,CARLINJ


In [35]:
adm_df1.loc[(adm_df1["PEOPLE_CODE_ID"]=="P000060480")]

,PEOPLE_CODE_ID,admission_status,create_date,status
1704,P000060480,1_Applied,2020-02-14,1
2269,P000060480,2_Accepted,2020-03-05,1
1747,P000060480,3_Deposited,2020-06-30,1
522,P000060480,4_Canceled,2020-06-02,1


In [36]:
df_deposits.loc[(df_deposits["PEOPLE_CODE_ID"]=="P000060480")]

,PEOPLE_CODE_ID,LAST_NAME,FIRST_NAME,MIDDLE_NAME,BIRTH_DATE,GOVERNMENT_ID,EMAIL_ADDRESS,ADDRESS_TYPE,ADDRESS_LINE_1,CITY,...,ACADEMIC_YEAR,ACADEMIC_TERM,ADMIT_YEAR,ADMIT_TERM,APP_STATUS_DATE,APP_STATUS,COLLEGE_ATTEND,REVISION_DATE,REVISION_TIME,REVISION_OPID
156,P000060480,Pierson,Hannah,,2000-06-29,064905850,hpierson@paulsmiths.edu,HOME,8003 E Main Rd,Leroy,...,2020,FALL,2020,FALL,2020-06-30,500,TRAN,2020-07-01,1900-01-01 11:34:46.187,CARLINJ


In [37]:
adm_df1.loc[(adm_df1["PEOPLE_CODE_ID"]=="P000059692")]

,PEOPLE_CODE_ID,admission_status,create_date,status
1426,P000059692,1_Applied,2020-02-14,1
2751,P000059692,2_Accepted,2020-03-26,1
3300,P000059692,3_Deposited,2020-05-18,1


In [38]:
df_deposits.loc[(df_deposits["PEOPLE_CODE_ID"]=="P000059692")]

,PEOPLE_CODE_ID,LAST_NAME,FIRST_NAME,MIDDLE_NAME,BIRTH_DATE,GOVERNMENT_ID,EMAIL_ADDRESS,ADDRESS_TYPE,ADDRESS_LINE_1,CITY,...,ACADEMIC_YEAR,ACADEMIC_TERM,ADMIT_YEAR,ADMIT_TERM,APP_STATUS_DATE,APP_STATUS,COLLEGE_ATTEND,REVISION_DATE,REVISION_TIME,REVISION_OPID
170,P000059692,Ripp,Aidan,Paul,2000-06-29,476374010,aripp@paulsmiths.edu,HOME,1402 Slate St.,Cloquet,...,2020,FALL,2020,FALL,2020-05-18,500,FTF,2020-06-29,1900-01-01 08:49:55.190,CARLINJ
